<a href="https://colab.research.google.com/github/dyscarnate/harvesting-optimization/blob/main/%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D1%83%D0%B1%D0%BE%D1%80%D0%BA%D0%B8_%D1%83%D1%80%D0%BE%D0%B6%D0%B0%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten
from tensorflow.keras.layers import UpSampling2D, Reshape, Conv2DTranspose
from tensorflow.keras.layers import Multiply, AveragePooling2D
from tensorflow.keras.layers import Concatenate, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Уборка от "поля"

In [ ]:
# поля
import numpy as np
import pandas as pd
import random
import math

field = [
    [ 0, 23,  4, 17, 36, 12],
    [23,  0,  9, 32, 13, 27],
    [ 4,  9,  0,  6,  8, 25],
    [17, 32,  6,  0, 43, 56],
    [36, 13,  8,  43, 0,  7],
    [12, 27, 25,  56, 7,  0]
]

df = pd.DataFrame(data=field)
df

,0,1,2,3,4,5
0,0,23,4,17,36,12
1,23,0,9,32,13,27
2,4,9,0,6,8,25
3,17,32,6,0,43,56
4,36,13,8,43,0,7
5,12,27,25,56,7,0


In [ ]:
# длина бота исходя из условий задачи
LEN_BOT = sum(df.shape) - 1
LEN_BEST = 5
LEN_POPUL = (LEN_BEST + math.comb(LEN_BEST, 2)) * 2 + LEN_BEST
MUT_PERCENT = 0.3
EPOCHS = 30

print(LEN_POPUL)
# пример бота
b = [0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0]

35


In [ ]:
# функция оценки
def loss(bot):
    S = 0
    for i in range(1,6):
        if bot.count(i) == 0:
            S += 40_000
        elif bot.count(i) != 1:
            S += 10_000 * 2**bot.count(i)
    
    if bot[0] != 0: S += 20_000
    if bot[-1] != 0: S += 20_000

    koord = [bot[i:i+2] for i in range(len(bot)-1)]
    for i in koord:
        S += df.iloc[i[0], i[1]]
    return S 

# новый бот
def new_bot(LEN_BOT):
  bot = []
  for i in range(LEN_BOT):
    bot.append(random.randint(0,5))
  return bot

# скрещивание
def hrest(bot1, bot2):
  child1 = bot1[:5] + bot2[5:]
  child2 = bot2[:5] + bot1[5:]
  return [child1, child2]

# мутация
def mutation(bot, MUT_PERCENT = 0.3):
  count_mut_idx = int(round(len(bot)*MUT_PERCENT, 0))
  mut_idx = random.sample(range(len(bot)), k = count_mut_idx)
  for i in mut_idx:
    bot[i] = random.randint(0,5)
  return bot

In [ ]:
popul = []
best = ()
for i in range(LEN_POPUL):
    popul.append(new_bot(LEN_BOT))

for ep in range(EPOCHS):

  # мутация
    for i in range(len(popul)):
        popul.append(mutation(popul[i], MUT_PERCENT))

  # оценка популяции
    old_popul = []
    for bot in popul:
        old_popul.append([bot, loss(bot)])

  # сортировка популяции
    old_popul.sort(key = lambda x: x[1])
    best = (ep, old_popul[0][1], old_popul[0][0])
    print(best)

  # отбор лучших
    popul = []
    for i in range(LEN_BEST):
        popul.append(old_popul[i][0])

  # скрещивание
    for m in range(LEN_POPUL):
        for p in range(LEN_POPUL):
            popul += hrest(popul[m], popul[p])


    
print(f'\n\n\n diagram:\n{best[2]} \n\n len path:\n{best[1]}')

(0, 100148, [0, 5, 0, 0, 0, 0, 3, 1, 5, 0, 4])
(1, 20108, [0, 5, 0, 0, 0, 0, 3, 1, 0, 2, 4])
(2, 166, [0, 5, 0, 0, 0, 4, 3, 1, 0, 2, 0])
(3, 126, [0, 5, 0, 0, 0, 0, 3, 1, 2, 4, 0])
(4, 98, [0, 5, 0, 0, 0, 0, 3, 1, 4, 2, 0])
(5, 98, [0, 5, 0, 0, 0, 0, 3, 1, 4, 2, 0])
(6, 98, [0, 0, 5, 0, 0, 0, 3, 1, 4, 2, 0])
(7, 98, [0, 5, 0, 0, 0, 0, 3, 1, 4, 2, 0])
(8, 91, [0, 0, 5, 0, 0, 0, 3, 2, 4, 1, 0])
(9, 91, [0, 0, 5, 0, 0, 0, 3, 2, 4, 1, 0])
(10, 91, [0, 0, 5, 0, 0, 0, 3, 2, 4, 1, 0])
(11, 91, [0, 0, 5, 0, 0, 0, 3, 2, 4, 1, 0])
(12, 89, [0, 0, 2, 0, 0, 0, 3, 1, 4, 5, 0])
(13, 79, [0, 0, 3, 0, 0, 0, 2, 1, 4, 5, 0])
(14, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(15, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(16, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(17, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(18, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(19, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(20, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(21, 64, [0, 0, 0, 0, 0, 3, 2, 1, 4, 5, 0])
(22, 64, [0, 0, 0, 0, 0, 3, 2, 1,